In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import sys
import importlib

# Determine root directory - go up from notebooks folder to portfolio_analyser
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Reload module to pick up any code changes without restarting kernel
if 'full_scanner' in sys.modules:
    import full_scanner
    importlib.reload(full_scanner)

from full_scanner import (
    get_sp500_tickers,
    get_nasdaq100_tickers,
    get_portfolio_tickers,
    scan_stocks,
    filter_buy_signals,
    filter_sell_signals,
    create_excel_output,
    create_portfolio_excel,
    create_pdf_report,
    create_best_trades_excel,
    create_best_trades_pdf,
    cleanup_old_scans
)

# Setup results directory
results_dir = ROOT / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print("[OK] Scanner modules loaded")
print(f"[OK] Results directory: {results_dir}")
print(f"[OK] Timestamp: {timestamp}")

[OK] Scanner modules loaded
[OK] Results directory: c:\workspace\portfolio_analyser\scanner_results
[OK] Timestamp: 20260110_2254


## Configuration

Set your scan parameters:
- **daily_bars**: Number of daily bars to analyze (default: 60)
- **weekly_bars**: Number of weekly bars to analyze (default: 52)
- **concurrency**: Number of parallel workers (default: 2, max recommended: 5)

In [2]:
# Scan parameters
DAILY_BARS = 60
WEEKLY_BARS = 52
CONCURRENCY = 2  # Be careful with higher values (rate limits)

print(f"Daily bars: {DAILY_BARS}")
print(f"Weekly bars: {WEEKLY_BARS}")
print(f"Concurrency: {CONCURRENCY}")

Daily bars: 60
Weekly bars: 52
Concurrency: 2


## Scan Portfolio Stocks

Quick scan of your portfolio stocks from stocks.txt. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [3]:
print("Loading portfolio tickers from stocks.txt...")
portfolio_tickers = get_portfolio_tickers()

if portfolio_tickers:
    print(f"Found {len(portfolio_tickers)} portfolio stocks\n")

    # Run scan
    portfolio_results = scan_stocks(
        portfolio_tickers,
        category="Portfolio",
        daily_bars=DAILY_BARS,
        weekly_bars=WEEKLY_BARS,
        concurrency=CONCURRENCY
    )

    if not portfolio_results.empty:
        # Get buy signals (all and quality-filtered)
        portfolio_buy_all = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD', quality_filter=False)
        portfolio_buy = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD', quality_filter=True)
        
        # Get sell signals (all bearish signals)
        portfolio_sell = filter_sell_signals(portfolio_results, quality_filter=False)
        
        # Display summary
        print(f"\n[STATS] Portfolio Results:")
        print(f"   [OK] {len(portfolio_buy)} buy signals (EXCELLENT/GOOD/OK)")
        if len(portfolio_buy_all) > len(portfolio_buy):
            print(f"   [WAIT] {len(portfolio_buy_all) - len(portfolio_buy)} extended/weak (wait)")
        print(f"   [SELL] {len(portfolio_sell)} sell signals (bearish)")
        
        # Display buy opportunities
        if not portfolio_buy.empty:
            print("\n[OK] BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
            display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
            if all(col in portfolio_buy.columns for col in display_cols):
                print(portfolio_buy[display_cols].to_string(index=False))
            else:
                print(portfolio_buy[['ticker', 'signal', 'current_price']].to_string(index=False))
        
        # Display sell signals
        if not portfolio_sell.empty:
            print("\n[SELL] SELL SIGNALS (Bearish - reduce exposure):")
            display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
            if all(col in portfolio_sell.columns for col in display_cols):
                print(portfolio_sell[display_cols].to_string(index=False))
            else:
                print(portfolio_sell[['ticker', 'signal', 'current_price']].to_string(index=False))
        
        # Save combined results
        if not portfolio_buy.empty or not portfolio_sell.empty:
            # Create portfolio subfolder
            portfolio_dir = results_dir / 'portfolio'
            portfolio_dir.mkdir(exist_ok=True)
            
            xlsx_path = portfolio_dir / f'portfolio_scanner_{timestamp}.xlsx'
            pdf_path = portfolio_dir / f'scanner_report_portfolio_{timestamp}.pdf'
            best_trades_xlsx = portfolio_dir / f'portfolio_best_trades_{timestamp}.xlsx'
            best_trades_pdf = portfolio_dir / f'portfolio_best_trades_{timestamp}.pdf'
            
            # Create standard reports with both buy and sell data
            create_excel_output(portfolio_buy, portfolio_sell, xlsx_path, category="Portfolio")
            create_pdf_report(portfolio_buy, portfolio_sell, pdf_path, timestamp, category="Portfolio")
            
            # Create best trades reports
            create_best_trades_excel(portfolio_buy, portfolio_sell, best_trades_xlsx, category="Portfolio")
            create_best_trades_pdf(portfolio_buy, portfolio_sell, best_trades_pdf, category="Portfolio")
            
            print(f"\n[OK] Excel: portfolio/{xlsx_path.name}")
            print(f"[OK] PDF: portfolio/{pdf_path.name}")
            print(f"[OK] Best Trades Excel: portfolio/{best_trades_xlsx.name}")
            print(f"[OK] Best Trades PDF: portfolio/{best_trades_pdf.name}")
else:
    print("[WARN] No portfolio tickers found in stocks.txt")

Loading portfolio tickers from stocks.txt...
[OK] Loaded 9 portfolio tickers from stocks.txt

Found 9 portfolio stocks

[SCAN] Scanning 9 Portfolio stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [1/9] TSLA   -> FULL HOLD + ADD      $445.01 | Entry: CAUTION    ⚠️ THIN
[OK] [2/9] NVDA   -> FULL HOLD + ADD      $184.86 | Entry: CAUTION    ⚠️ THIN
[OK] [3/9] ASML   -> FULL HOLD + ADD      $1,273.88 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [6/9] MRVL   -> FULL HOLD + ADD      $83.22 | Entry: GOOD       ✓ SAFE ENTRY

[OK] Scan complete: 9 analyzed, 4 FULL HOLD + ADD signals found


[STATS] Portfolio Results:
   [OK] 1 buy signals (EXCELLENT/GOOD/OK)
   [WAIT] 3 extended/weak (wait)
   [SELL] 3 sell signals (bearish)

[OK] BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):
ticker          signal  current_price buy_quality        s1
  MRVL FULL HOLD + ADD          83.22        GOOD 81.121628

[SELL] SELL SIGNALS (Bearish - reduce exposure):
 ticker   

## Scan S&P 500

Full scan of ~500 stocks. Takes 5-10 minutes depending on cache. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [4]:
print("Fetching S&P 500 ticker list...")
sp500_tickers = get_sp500_tickers()
print(f"Found {len(sp500_tickers)} S&P 500 stocks\n")

# Run scan
sp500_results = scan_stocks(
    sp500_tickers,
    category="S&P 500",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not sp500_results.empty:
    # Get buy signals (all and quality-filtered)
    sp500_buy_all = filter_buy_signals(sp500_results, 'FULL HOLD + ADD', quality_filter=False)
    sp500_buy = filter_buy_signals(sp500_results, 'FULL HOLD + ADD', quality_filter=True)
    
    # Get sell signals (all bearish signals, matching portfolio_analysis logic)
    sp500_sell = filter_sell_signals(sp500_results, quality_filter=False)
    
    # Display summary
    print(f"\n[STATS] S&P 500 Results:")
    print(f"   [OK] {len(sp500_buy)} buy signals (EXCELLENT/GOOD/OK)")
    if len(sp500_buy_all) > len(sp500_buy):
        print(f"   [WAIT] {len(sp500_buy_all) - len(sp500_buy)} extended/weak (wait)")
    print(f"   [SELL] {len(sp500_sell)} sell signals (bearish)")
    
    # Display buy opportunities
    if not sp500_buy.empty:
        print("\n[OK] TOP BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
        display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
        if all(col in sp500_buy.columns for col in display_cols):
            print(sp500_buy[display_cols].head(10).to_string(index=False))
        else:
            print(sp500_buy[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Display sell signals (matches portfolio_analysis behavior)
    if not sp500_sell.empty:
        print("\n[SELL] SELL SIGNALS (Bearish - reduce exposure):")
        display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
        if all(col in sp500_sell.columns for col in display_cols):
            print(sp500_sell[display_cols].head(10).to_string(index=False))
        else:
            print(sp500_sell[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Save combined results if we have either buy or sell signals
    if not sp500_buy.empty or not sp500_sell.empty:
        # Create sp500 subfolder
        sp500_dir = results_dir / 'sp500'
        sp500_dir.mkdir(exist_ok=True)
        
        xlsx_path = sp500_dir / f'sp500_analysis_{timestamp}.xlsx'
        pdf_path = sp500_dir / f'scanner_report_sp500_{timestamp}.pdf'
        best_trades_xlsx = sp500_dir / f'sp500_best_trades_{timestamp}.xlsx'
        best_trades_pdf = sp500_dir / f'sp500_best_trades_{timestamp}.pdf'
        
        # Create standard reports with both buy and sell data
        create_excel_output(sp500_buy, sp500_sell, xlsx_path, category="S&P 500")
        create_pdf_report(sp500_buy, sp500_sell, pdf_path, timestamp, category="S&P 500")
        
        # Create best trades reports
        create_best_trades_excel(sp500_buy, sp500_sell, best_trades_xlsx, category="S&P 500")
        create_best_trades_pdf(sp500_buy, sp500_sell, best_trades_pdf, category="S&P 500")
        
        print(f"\n[OK] Excel: sp500/{xlsx_path.name}")
        print(f"[OK] PDF: sp500/{pdf_path.name}")
        print(f"[OK] Best Trades Excel: sp500/{best_trades_xlsx.name}")
        print(f"[OK] Best Trades PDF: sp500/{best_trades_pdf.name}")
else:
    print("[WARN] No results from S&P 500 scan")

Fetching S&P 500 ticker list...
[OK] Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

[SCAN] Scanning 503 S&P 500 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [4/503] ABBV   -> FULL HOLD + ADD      $220.08 | Entry: OK         🎯 ACCEPTABLE
[OK] [8/503] AFL    -> FULL HOLD + ADD      $109.24 | Entry: GOOD       ✓ SAFE ENTRY
[OK] [10/503] A      -> FULL HOLD + ADD      $148.52 | Entry: OK         🎯 ACCEPTABLE
[OK] [14/503] ALB    -> FULL HOLD + ADD      $161.29 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [19/503] GOOGL  -> FULL HOLD + ADD      $328.57 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [20/503] ALL    -> FULL HOLD + ADD      $212.21 | Entry: OK         🎯 ACCEPTABLE
[OK] [21/503] GOOG   -> FULL HOLD + ADD      $329.14 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [22/503] AMZN   -> FULL HOLD + ADD      $247.38 | Entry: CAUTION    ⚠️ THIN
[OK] [27/503] AXP    -> FULL HOLD + ADD      $375.61 | Entry: CAUTION    ⚠️ THIN
[OK] [28/503] AIG    -> FULL 

## Scan NASDAQ 100

Scan ~100 tech-heavy stocks. Takes 2-5 minutes. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [5]:
print("Fetching NASDAQ 100 ticker list...")
nasdaq100_tickers = get_nasdaq100_tickers()
print(f"Found {len(nasdaq100_tickers)} NASDAQ 100 stocks\n")

# Run scan
nasdaq100_results = scan_stocks(
    nasdaq100_tickers,
    category="NASDAQ 100",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not nasdaq100_results.empty:
    # Get buy signals (all and quality-filtered)
    nasdaq100_buy_all = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD', quality_filter=False)
    nasdaq100_buy = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD', quality_filter=True)
    
    # Get sell signals (all bearish signals, matching portfolio_analysis logic)
    nasdaq100_sell = filter_sell_signals(nasdaq100_results, quality_filter=False)
    
    # Display summary
    print(f"\n[STATS] NASDAQ 100 Results:")
    print(f"   [OK] {len(nasdaq100_buy)} buy signals (EXCELLENT/GOOD/OK)")
    if len(nasdaq100_buy_all) > len(nasdaq100_buy):
        print(f"   [WAIT] {len(nasdaq100_buy_all) - len(nasdaq100_buy)} extended/weak (wait)")
    print(f"   [SELL] {len(nasdaq100_sell)} sell signals (bearish)")
    
    # Display buy opportunities
    if not nasdaq100_buy.empty:
        print("\n[OK] TOP BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
        display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
        if all(col in nasdaq100_buy.columns for col in display_cols):
            print(nasdaq100_buy[display_cols].head(10).to_string(index=False))
        else:
            print(nasdaq100_buy[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Display sell signals (matches portfolio_analysis behavior)
    if not nasdaq100_sell.empty:
        print("\n[SELL] SELL SIGNALS (Bearish - reduce exposure):")
        display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
        if all(col in nasdaq100_sell.columns for col in display_cols):
            print(nasdaq100_sell[display_cols].head(10).to_string(index=False))
        else:
            print(nasdaq100_sell[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Save combined results if we have either buy or sell signals
    if not nasdaq100_buy.empty or not nasdaq100_sell.empty:
        # Create nasdaq100 subfolder
        nasdaq100_dir = results_dir / 'nasdaq100'
        nasdaq100_dir.mkdir(exist_ok=True)
        
        xlsx_path = nasdaq100_dir / f'nasdaq100_analysis_{timestamp}.xlsx'
        pdf_path = nasdaq100_dir / f'scanner_report_nasdaq100_{timestamp}.pdf'
        best_trades_xlsx = nasdaq100_dir / f'nasdaq100_best_trades_{timestamp}.xlsx'
        best_trades_pdf = nasdaq100_dir / f'nasdaq100_best_trades_{timestamp}.pdf'
        
        # Create standard reports with both buy and sell data
        create_excel_output(nasdaq100_buy, nasdaq100_sell, xlsx_path, category="NASDAQ 100")
        create_pdf_report(nasdaq100_buy, nasdaq100_sell, pdf_path, timestamp, category="NASDAQ 100")
        
        # Create best trades reports
        create_best_trades_excel(nasdaq100_buy, nasdaq100_sell, best_trades_xlsx, category="NASDAQ 100")
        create_best_trades_pdf(nasdaq100_buy, nasdaq100_sell, best_trades_pdf, category="NASDAQ 100")
        
        print(f"\n[OK] Excel: nasdaq100/{xlsx_path.name}")
        print(f"[OK] PDF: nasdaq100/{pdf_path.name}")
        print(f"[OK] Best Trades Excel: nasdaq100/{best_trades_xlsx.name}")
        print(f"[OK] Best Trades PDF: nasdaq100/{best_trades_pdf.name}")
else:
    print("[WARN] No results from NASDAQ 100 scan")

Fetching NASDAQ 100 ticker list...
[OK] Loaded 101 NASDAQ 100 tickers

Found 101 NASDAQ 100 stocks

[SCAN] Scanning 101 NASDAQ 100 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [5/101] GOOGL  -> FULL HOLD + ADD      $328.57 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [6/101] GOOG   -> FULL HOLD + ADD      $329.14 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [7/101] AMZN   -> FULL HOLD + ADD      $247.38 | Entry: CAUTION    ⚠️ THIN
[OK] [9/101] AMGN   -> FULL HOLD + ADD      $326.10 | Entry: EXCELLENT  ✓ SAFE ENTRY
[OK] [10/101] ADI    -> FULL HOLD + ADD      $300.93 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [11/101] AAPL   -> FULL HOLD + ADD      $259.37 | Entry: CAUTION    ⚠️ THIN
[OK] [12/101] AMAT   -> FULL HOLD + ADD      $301.18 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [13/101] APP    -> FULL HOLD + ADD      $647.72 | Entry: CAUTION    ⚠️ THIN
[OK] [15/101] ASML   -> FULL HOLD + ADD      $1,273.88 | Entry: CAUTION    ⚠️ EXTENDED
[OK] [16/101] AZN    -> 

## Cleanup Old Scan Files

Archive old scan results to keep the scanner_results folder clean. Keeps the 2 most recent files per category (Portfolio, S&P 500, NASDAQ 100).

In [6]:
print("\n" + "="*80)
print("CLEANUP - Archiving Old Scan Files")
print("="*80)

# Keep 1 most recent scan per category (4 files per scan: 2 standard + 2 best trades)
# Archives older files, deletes archives older than 60 days
cleanup_old_scans(results_dir, max_files=1, archive_retention_days=60)

print("\n[OK] Cleanup complete")


CLEANUP - Archiving Old Scan Files
  📦 Archived (sp500): sp500_analysis_20260110_2249.xlsx
  📦 Archived (sp500): scanner_report_sp500_20260110_2249.pdf
  📦 Archived (sp500): sp500_best_trades_20260110_2249.xlsx
  📦 Archived (sp500): sp500_best_trades_20260110_2249.pdf
  📦 Archived (nasdaq100): nasdaq100_analysis_20260110_2249.xlsx
  📦 Archived (nasdaq100): scanner_report_nasdaq100_20260110_2249.pdf
  📦 Archived (nasdaq100): nasdaq100_best_trades_20260110_2249.xlsx
  📦 Archived (nasdaq100): nasdaq100_best_trades_20260110_2249.pdf
  📦 Archived (portfolio): portfolio_scanner_20260110_2249.xlsx
  📦 Archived (portfolio): scanner_report_portfolio_20260110_2249.pdf
  📦 Archived (portfolio): portfolio_best_trades_20260110_2249.xlsx
  📦 Archived (portfolio): portfolio_best_trades_20260110_2249.pdf
  ✅ Archived 12 file(s), kept 1 most recent

[OK] Cleanup complete
